#### Импорты

In [14]:
import momepy
import osmnx as ox
import pandas as pd
import geopandas as gpd
from metrics.grade_territory import grade_territory
from graph_builder.graphbuilder import get_graph_from_polygon, get_frame, assign_city_names_to_nodes

#### Загрузка графа

In [9]:
lo_polygon = ox.geocode_to_gdf('R176095', by_osmid=True).to_crs(epsg=32636)
spb_polygon = ox.geocode_to_gdf('R337422', by_osmid=True).to_crs(epsg=32636).buffer(3000)
city = lo_polygon.union(spb_polygon).to_crs(epsg=4326) #  get lo polygon
russia = ox.geocode_to_gdf("Russia")

citygraph = get_graph_from_polygon(city, crs=32636,country_polygon=russia)

d:\IT\GitHub\transport_frames\.venv\Lib\site-packages\osmnx\_overpass.py:254: UserWarning: This area is 52 times your configured Overpass max query area size. It will automatically be divided up into multiple sub-queries accordingly. This may take a long time.
  multi_poly_proj = utils_geo._consolidate_subdivide_geometry(poly_proj)


#### Выделение каркаса

In [12]:
'''
IDK who is it 
regions = regions[regions['ISO3166-2']!='RU-CHU']
'''

regions = gpd.read_file('data/russia.geojson') #  get regions
regions = regions[regions['ISO3166-2']!='RU-CHU']
regions = regions.to_crs(city.crs)

frame = get_frame(citygraph, regions, city)

C:\Users\romab\AppData\Local\Temp\ipykernel_14268\4137021309.py:10: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  frame = get_frame(citygraph, regions, city)
d:\IT\GitHub\transport_frames\transport_frames\graph_builder\graphbuilder.py:463: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '[0.04]' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  n.loc[(n['nodeID']==path[j]),'weight']+= weight
d:\IT\GitHub\transport_frames\transport_frames\graph_builder\graphbuilder.py:464: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '[0.04]' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  e.loc[(e['node_start'] == path[j]) & (e['node_end'] == path[j+1]), 'weight'] +=weight


#### Перенос точек на граф

In [13]:
cities = gpd.read_file('data/18_centers.geojson')
frame = assign_city_names_to_nodes(cities.to_crs(frame.graph['crs']), momepy.nx_to_gdf(frame)[0], frame, name_col='name18', max_distance=1200)

#### Оценка территории

In [15]:
p1 = gpd.read_file('data/Аэродром лодейнопольское поселение.geojson')
p2 = gpd.read_file('data/Аэродром Сиверск .geojson')
p3 = gpd.read_file('data/project Светогорского поселения.geojson')
p4 = gpd.read_file('data/project Шлиссельбург.geojson')
neudobiya = pd.concat([p1,p2,p3,p4])

In [16]:
n_grade = grade_territory(neudobiya, frame)